## 🎓 Persona : Léa, jeune investisseuse étudiante

**Profil :**
- 👩 24 ans, diplômée de l'EM Lyon
- 💼 Première expérience professionnelle après 2 ans d'alternance
- 💰 Aide parentale pour le financement + épargne personnelle (~15 000 €)
- 🎯 Objectif : réaliser un **premier investissement locatif** dans une **ville étudiante dynamique**

---

### 💡 Objectif d'investissement
> Trouver le **meilleur investissement locatif étudiant** possible avec un **budget global de 200 000 €**,  
> en ciblant un **studio à Lille**, tout en comparant brièvement avec un **T1 à Angers ou Nancy**.

---

### 💰 Hypothèses financières
| Élément | Montant estimé |
|----------|----------------|
| Prix d'achat visé | 160 000 – 180 000 € |
| Apport personnel | 15 000 € |
| Prêt immobilier estimé | 180 000 € sur 20 ans |
| Budget total (frais inclus) | **≈ 200 000 €** |
| Objectif de rentabilité brute | **≥ 5 %** |

---

### 🏙️ Cibles principales
| Ville | Type de bien | Prix moyen au m² | Loyer moyen mensuel | Observations |
|-------|---------------|------------------|---------------------|---------------|
| **Lille** | Studio (20–25 m²) | ~4 500 €/m² | 550–600 € | Marché étudiant tendu, forte demande locative |
| **Angers** | T1 (25–30 m²) | ~3 200 €/m² | 450–500 € | Ville très dynamique, bonne rentabilité brute |
| **Nancy** | T1 (25–30 m²) | ~2 800 €/m² | 420–470 € | Marché abordable, bon rapport prix/rentabilité |

---

### 🔍 Besoins data de Léa
- Identifier **les quartiers les plus rentables** à Lille (ou dans des villes comparables)
- Comparer avec **la rentabilité moyenne en France**
- Analyser l'**évolution du prix au m² et des loyers étudiants** depuis 5 ans
- Calculer la **rentabilité locative brute et nette** par quartier
- Visualiser les **zones à forte concentration étudiante**
- Fournir une **recommandation finale : "où investir avec 200k€ ?"**
- Évaluer le **taux de vacance locative** par quartier pour anticiper les périodes creuses (notamment l'été où les étudiants quittent les logements)
- Analyser la **proximité des transports en commun** et des universités/grandes écoles pour identifier les zones les plus attractives pour les étudiants
- Estimer les **charges de copropriété moyennes** par type de bien et par quartier pour affiner le calcul de rentabilité nette
- Identifier les **opportunités de biens nécessitant des travaux** (décote à l'achat) pour maximiser la plus-value à long terme

---

### 🧭 Objectif du notebook
Créer un outil interactif permettant à Léa de :
1. Comparer la rentabilité d'un **studio à Lille** avec celle d'un **T1 à Angers ou Nancy**  
2. Explorer visuellement les **zones à potentiel locatif élevé**  
3. Obtenir une **recommandation automatique** en fonction de son budget et de ses préférences

## Import des bibliothèques ##

In [35]:
import pandas as pd 
import plotly.express as px
import time
import nbformat

Évaluer le taux de vacance locative en France, et les zones propices à un taux élevé

Constante

In [36]:
BDD = "https://huggingface.co/datasets/analysedonneesfoncieresdata/analyse_fonciere_data/resolve/main"

Lecture fichiers Excel

In [37]:
df_vac = pd.read_excel(f"{BDD}/insee_rp_hist_1968.xlsx", header=1)
df_pop = pd.read_excel(f"{BDD}/POPULATION_MUNICIPALE_COMMUNES_FRANCE_lucien.xlsx")
df_communes = pd.read_csv(f"{BDD}/communes-france-2025.csv", sep=",")  # adapte le chemin si nécessaire

C:\Users\rivat\AppData\Local\Temp\ipykernel_32676\1433042141.py:3: DtypeWarning:

Columns (1,12,14,16,23,24) have mixed types. Specify dtype option on import or set low_memory=False.



In [38]:
df_communes_coords = df_communes[[
    "code_insee",       # code INSEE pour faire le merge
    "code_postal",      # code postal
    "latitude_centre",  # latitude du centre de la commune
    "longitude_centre"  # longitude du centre de la commune
]]

Mise en forme des colonnes dans les fichiers

In [39]:
df_vac.columns = ["code_commune", "nom_commune", "annee", "part_log_vacant", "col5", "col6", "col7", "col8"]
df_vac = df_vac[["code_commune", "nom_commune", "annee", "part_log_vacant"]]

df_pop.columns = ["objectid", "reg", "dep", "cv", "codgeo",	"libgeo", "p21_pop"]

df_vac = df_vac.dropna(subset=["part_log_vacant"])
df_vac["part_log_vacant"] = pd.to_numeric(df_vac["part_log_vacant"], errors="coerce")
df_vac = df_vac.dropna(subset=["part_log_vacant"])

df_pop["codgeo"] = df_pop["codgeo"].astype(str)
df_vac["code_commune"] = df_vac["code_commune"].astype(str)

Dataframe des villes de plus de 100 000 habitants classées dans l'ordre croissant par rapport au taux de logement vacant.

In [40]:
df_vac = df_vac.sort_values(by="part_log_vacant", ascending=True)

df_pop_100k = df_pop[df_pop["p21_pop"] > 100000]

df_vac_100k = df_vac.merge(
    df_pop_100k[["codgeo", "p21_pop"]],
    left_on="code_commune",
    right_on="codgeo",
    how="inner"
)

df_communes_coords["code_insee"] = df_communes_coords["code_insee"].astype(str)
df_vac_100k["code_commune"] = df_vac_100k["code_commune"].astype(str)

df_vac_100k = df_vac_100k[["code_commune", "nom_commune", "annee", "part_log_vacant", "p21_pop"]]

C:\Users\rivat\AppData\Local\Temp\ipykernel_32676\1603771164.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Affichage sur la map

In [41]:
# --- Garder uniquement l'année la plus récente pour chaque commune ---
df_vac_100k_recent = df_vac_100k.sort_values("annee", ascending=False) \
                                 .groupby("code_commune", as_index=False) \
                                 .first()  # prend la ligne la plus récente pour chaque commune

# --- Merge avec les coordonnées ---
df_vac_map = df_vac_100k_recent.merge(
    df_communes_coords,
    left_on="code_commune",
    right_on="code_insee",
    how="left"
).dropna(subset=["latitude_centre", "longitude_centre"])

# --- Affichage carte Plotly ---
import plotly.express as px

fig = px.scatter_mapbox(
    df_vac_map,
    lat="latitude_centre",
    lon="longitude_centre",
    size="part_log_vacant",
    color="part_log_vacant",
    hover_name="nom_commune",
    hover_data={
        "p21_pop": True,
        "part_log_vacant": True,
        "code_postal": True,
        "annee": True
    },
    zoom=5,
    height=500,
    color_continuous_scale="OrRd"
)
# --- Top 10 communes avec le plus petit taux de logements vacants ---
top10_low_vac = df_vac_map.sort_values("part_log_vacant", ascending=True).head(20)

# --- Histogramme ---
fig_hist = px.bar(
    top10_low_vac,
    x="nom_commune",
    y="part_log_vacant",
    text="part_log_vacant",
    hover_data={"p21_pop": True, "code_postal": True, "annee": True},
    labels={"part_log_vacant": "Taux logements vacants", "nom_commune": "Commune"},
    title="Top 10 communes avec le plus petit taux de logements vacants",
    color="part_log_vacant",
    color_continuous_scale="Blues",
    height=600
)

fig.update_layout(mapbox_style="open-street-map", margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

fig_hist.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig_hist.show()

C:\Users\rivat\AppData\Local\Temp\ipykernel_32676\73503755.py:17: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



Etude rentabilité par quartier à Rennes

In [50]:
import requests, zipfile, io
df = pd.read_csv(f"{BDD}/rennes_quartiers.csv.txt", sep=",")
r = requests.get(f"{BDD}/data_loyer.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
fichier_csv = [f for f in z.namelist() if f.endswith(".csv")][0]
df2 = pd.read_csv(z.open(fichier_csv), encoding="latin-1", sep=";")
df.columns = df.columns.str.strip()
df2.columns = df2.columns.str.strip()
display(df2)
display(df)

,Observatory,Data_year,agglomeration,Zone_calcul,Zone_complementaire,Type_habitat,epoque_construction_local,epoque_construction_homogene,anciennete_locataire_local,anciennete_locataire_homogene,...,loyer_mensuel_1_decile,loyer_mensuel_1_quartile,loyer_mensuel_median,loyer_mensuel_3_quartile,loyer_mensuel_9_decile,moyenne_loyer_mensuel,surface_moyenne,nombre_observations,nombre_logements,methodologie_production
0,B6300,2022,L6300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,345.0,410.0,503.0,648.0,840.0,556.0,63.0,10753.0,46756.0,Estimation directe
1,B6300,2022,L6300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,350.0,450.0,500.0,600.0,732.0,526.0,64.0,250.0,2952.0,Estimation directe
2,B6300,2022,L6300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,505.0,600.0,750.0,900.0,1072.0,772.0,98.0,2029.0,13250.0,Estimation directe
3,B6300,2022,L6300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,550.0,670.0,840.0,945.0,1146.0,835.0,106.0,638.0,6582.0,Estimation directe
4,B6300,2022,L6300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,289.0,315.0,348.0,380.0,408.0,348.0,27.0,2175.0,6798.0,Estimation directe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1927,B6300,2022,L6300,L6300.1.04,NaN,Appartement,5. Après 2005,NaN,NaN,NaN,...,410.0,430.0,465.0,485.0,530.0,463.0,44.0,92.0,272.0,Estimation directe
1928,B6300,2022,L6300,L6300.1.04,NaN,Appartement,5. Après 2005,NaN,NaN,NaN,...,480.0,500.0,580.0,610.0,623.0,562.0,63.0,67.0,368.0,Estimation directe
1929,B6300,2022,L6300,L6300.1.04,NaN,Appartement,5. Après 2005,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,NaN,Estimation directe
1930,B6300,2022,L6300,L6300.1.04,NaN,Appartement,5. Après 2005,NaN,1. Moins de 1 an,NaN,...,407.0,463.0,539.0,680.0,750.0,568.0,60.0,62.0,NaN,Estimation directe


,Quartier,Type de bien,Statut,Nb de pièces,Surface (m²),Prix au m² (€),Prix total estimé (€)
0,Thabor-Saint Helier,Appartement,ancien,1,30,4327,129810
1,Thabor-Saint Helier,Appartement,neuf,1,30,5897,176910
2,Thabor-Saint Helier,Appartement,ancien,2,45,4327,194715
3,Thabor-Saint Helier,Appartement,neuf,2,45,5897,265365
4,Thabor-Saint Helier,Appartement,ancien,3,65,4327,281255
...,...,...,...,...,...,...,...
163,Le Blosne,Maison,neuf,3,90,2734,246060
164,Le Blosne,Maison,ancien,4,110,2006,220660
165,Le Blosne,Maison,neuf,4,110,2734,300740
166,Le Blosne,Maison,ancien,5,130,2006,260780


In [51]:
df['Nb de pièces'] = df['Nb de pièces'].astype(int)
df['Surface (m²)'] = df['Surface (m²)'].astype(float)
df['Prix au m² (€)'] = df['Prix au m² (€)'].astype(float)
df['Prix total estimé (€)'] = df['Prix total estimé (€)'].astype(float)

In [52]:
df_etudiants = df[(df['Type de bien'] == 'Appartement') & (df['Surface (m²)'] < 50)]
df2 = df2[((df2['Type_habitat'] == 'Appartement') | (df2['nombre_pieces_homogene'] == 'Appart 1P') | (df2['nombre_pieces_homogene'] == 'Appart 2P')) & (df2['surface_moyenne'] < 50)]
display(df_etudiants)
display(df2)

,Quartier,Type de bien,Statut,Nb de pièces,Surface (m²),Prix au m² (€),Prix total estimé (€)
0,Thabor-Saint Helier,Appartement,ancien,1,30.0,4327.0,129810.0
1,Thabor-Saint Helier,Appartement,neuf,1,30.0,5897.0,176910.0
2,Thabor-Saint Helier,Appartement,ancien,2,45.0,4327.0,194715.0
3,Thabor-Saint Helier,Appartement,neuf,2,45.0,5897.0,265365.0
14,Atalante Beaulieu,Appartement,ancien,1,30.0,4063.0,121890.0
15,Atalante Beaulieu,Appartement,neuf,1,30.0,5536.0,166080.0
16,Atalante Beaulieu,Appartement,ancien,2,45.0,4063.0,182835.0
17,Atalante Beaulieu,Appartement,neuf,2,45.0,5536.0,249120.0
28,Centre,Appartement,ancien,1,30.0,3932.0,117960.0
29,Centre,Appartement,neuf,1,30.0,5357.0,160710.0


,Observatory,Data_year,agglomeration,Zone_calcul,Zone_complementaire,Type_habitat,epoque_construction_local,epoque_construction_homogene,anciennete_locataire_local,anciennete_locataire_homogene,...,loyer_mensuel_1_decile,loyer_mensuel_1_quartile,loyer_mensuel_median,loyer_mensuel_3_quartile,loyer_mensuel_9_decile,moyenne_loyer_mensuel,surface_moyenne,nombre_observations,nombre_logements,methodologie_production
4,B6300,2022,L6300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,289.0,315.0,348.0,380.0,408.0,348.0,27.0,2175.0,6798.0,Estimation directe
5,B6300,2022,L6300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,360.0,400.0,450.0,495.0,534.0,445.0,44.0,3557.0,12422.0,Estimation directe
178,B6300,2022,L6300,NaN,NaN,Appartement,NaN,NaN,NaN,NaN,...,289.0,315.0,348.0,380.0,408.0,348.0,27.0,2175.0,6798.0,Estimation directe
179,B6300,2022,L6300,NaN,NaN,Appartement,NaN,NaN,NaN,NaN,...,360.0,400.0,450.0,495.0,534.0,445.0,44.0,3557.0,12422.0,Estimation directe
182,B6300,2022,L6300,NaN,NaN,Appartement,NaN,NaN,NaN,NaN,...,289.0,315.0,348.0,380.0,408.0,348.0,27.0,2175.0,6798.0,Estimation directe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1871,B6300,2022,L6300,L6300.1.04,NaN,Appartement,NaN,2. Entre 1946-1970,NaN,NaN,...,330.0,362.0,375.0,454.0,491.0,402.0,48.0,65.0,383.0,Estimation directe
1892,B6300,2022,L6300,L6300.1.04,NaN,Appartement,NaN,5. Après 2005,NaN,NaN,...,410.0,430.0,465.0,485.0,530.0,463.0,44.0,92.0,272.0,Estimation directe
1899,B6300,2022,L6300,L6300.1.04,NaN,Appartement,1. Avant 1946,NaN,NaN,NaN,...,320.0,353.0,400.0,430.0,500.0,400.0,46.0,142.0,609.0,Estimation directe
1906,B6300,2022,L6300,L6300.1.04,NaN,Appartement,2. Entre 1946-1970,NaN,NaN,NaN,...,330.0,362.0,375.0,454.0,491.0,402.0,48.0,65.0,383.0,Estimation directe


In [53]:
df_etudiants = df_etudiants.sort_values(by="Prix au m² (€)")
display(df_etudiants)

,Quartier,Type de bien,Statut,Nb de pièces,Surface (m²),Prix au m² (€),Prix total estimé (€)
154,Le Blosne,Appartement,ancien,1,30.0,2006.0,60180.0
156,Le Blosne,Appartement,ancien,2,45.0,2006.0,90270.0
157,Le Blosne,Appartement,neuf,2,45.0,2734.0,123030.0
155,Le Blosne,Appartement,neuf,1,30.0,2734.0,82020.0
142,Bréquigny,Appartement,ancien,2,45.0,3119.0,140355.0
140,Bréquigny,Appartement,ancien,1,30.0,3119.0,93570.0
126,Villejean-Beauregard,Appartement,ancien,1,30.0,3199.0,95970.0
128,Villejean-Beauregard,Appartement,ancien,2,45.0,3199.0,143955.0
112,Maurepas-Patton,Appartement,ancien,1,30.0,3437.0,103110.0
114,Maurepas-Patton,Appartement,ancien,2,45.0,3437.0,154665.0


In [ ]:
rentabilite = loyer / prix d'achat au metre carre